# WaveMesh-Diff - Google Colab Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HoangNguyennnnnnn/WaveMeshDf/blob/main/colab_quickstart.ipynb)

**3D Mesh Generation using Diffusion Models in Wavelet Domain**

---

## ⚡ Quick Overview

This notebook demonstrates:
1. ✅ Setup WaveMesh-Diff in Google Colab
2. 🧪 Test all 4 modules (Wavelet, U-Net, Diffusion, Multi-view)
3. 📊 Visualize sparse wavelet representation
4. 🎨 Run quick demos

**Estimated time: 10-15 minutes**

---

## 🚀 Setup

### 1. Clone Repository

In [ ]:
!git clone https://github.com/HoangNguyennnnnnn/WaveMeshDf.git
%cd WaveMeshDf

### 2. Install Dependencies

In [ ]:
# Cài dependencies cơ bản
!pip install -q PyWavelets trimesh matplotlib rtree scipy

# PyTorch thường đã có sẵn trong Colab
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Cài thêm scikit-image cho marching cubes
!pip install -q scikit-image

print("✅ All dependencies installed!")

In [ ]:
# Quick verification - check imports work
try:
    import pywt
    import trimesh
    import matplotlib
    from skimage import measure
    print("✅ PyWavelets:", pywt.__version__)
    print("✅ Trimesh:", trimesh.__version__)
    print("✅ Matplotlib:", matplotlib.__version__)
    print("✅ scikit-image: OK")
    print("\n🎉 All core dependencies ready!")
except ImportError as e:
    print(f"❌ Missing dependency: {e}")
    print("Run the install cells above to fix this.")

### 3. Optional: Install Advanced Features

In [ ]:
# Cài transformers cho DINOv2 (tùy chọn - cải thiện quality)
!pip install -q transformers huggingface_hub

# Login HuggingFace (cần token từ https://huggingface.co/settings/tokens)
# from huggingface_hub import login
# login(token="your_token_here")

---

## 🧪 Test Installation

In [ ]:
# Test tất cả modules
!python test_all_modules.py

**Note:** Nếu gặp lỗi import, restart runtime và chạy lại từ đầu.

**Kỳ vọng:**
```
Results: 4/4 modules passed
  Module A ✅ PASS
  Module B ✅ PASS
  Module C ✅ PASS
  Module D ✅ PASS
```

---

## 📊 Quick Demo

### Module A: Wavelet Transform

In [ ]:
from data.wavelet_utils import mesh_to_sdf_simple, sdf_to_sparse_wavelet, sparse_wavelet_to_sdf
import trimesh
import numpy as np
import matplotlib.pyplot as plt

# Tạo mesh mẫu
mesh = trimesh.creation.box(extents=[1, 1, 1])
print(f"Mesh: {len(mesh.vertices)} vertices, {len(mesh.faces)} faces")

# Chuyển sang SDF
sdf = mesh_to_sdf_simple(mesh, resolution=32)
print(f"SDF shape: {sdf.shape}")

# Wavelet transform - trả về dictionary
sparse_data = sdf_to_sparse_wavelet(sdf, threshold=0.01)
print(f"Sparse indices: {sparse_data['indices'].shape}")
print(f"Sparse features: {sparse_data['features'].shape}")

# Calculate sparsity
total_elements = 32 ** 3
non_zero = len(sparse_data['features'])
sparsity = 100 * (1 - non_zero / total_elements)
print(f"Sparsity: {sparsity:.1f}%")

# Reconstruct
sdf_recon = sparse_wavelet_to_sdf(sparse_data)
mse = np.mean((sdf - sdf_recon) ** 2)
print(f"Reconstruction MSE: {mse:.6f}")

### Visualize SDF

In [ ]:
# Visualize SDF slice
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(sdf[16, :, :], cmap='RdBu')
axes[0].set_title('Original SDF (slice)')
axes[0].axis('off')

axes[1].imshow(sdf_recon[16, :, :], cmap='RdBu')
axes[1].set_title('Reconstructed SDF')
axes[1].axis('off')

diff = np.abs(sdf - sdf_recon)
axes[2].imshow(diff[16, :, :], cmap='hot')
axes[2].set_title(f'Error (MSE={mse:.6f})')
axes[2].axis('off')

plt.tight_layout()
plt.show()

---

### Module D: Multi-view Encoder

In [ ]:
from models import create_multiview_encoder
import torch

# Tạo encoder
encoder = create_multiview_encoder(preset='small')
print(f"Encoder created: {sum(p.numel() for p in encoder.parameters()):,} params")

# Test với random data
batch_size = 2
num_views = 4
images = torch.randn(batch_size, num_views, 3, 224, 224)
poses = torch.randn(batch_size, num_views, 3, 4)

# Forward pass
with torch.no_grad():
    conditioning = encoder(images, poses)

print(f"Input images: {images.shape}")
print(f"Input poses: {poses.shape}")
print(f"Output conditioning: {conditioning.shape}")
print("✅ Multi-view encoder working!")

---

### Module B + C: U-Net + Diffusion

In [ ]:
from models import WaveMeshUNet, GaussianDiffusion

# Tạo U-Net
unet = WaveMeshUNet(
    in_channels=1,
    encoder_channels=[16, 32, 64],
    decoder_channels=[64, 32, 16],
    time_emb_dim=128,
    use_attention=True,
    context_dim=384  # Match Module D output
)
print(f"U-Net: {sum(p.numel() for p in unet.parameters()):,} params")

# Tạo Diffusion
diffusion = GaussianDiffusion(
    timesteps=1000,
    beta_schedule='linear'
)
print(f"Diffusion: {diffusion.timesteps} timesteps")
print(f"Beta range: [{diffusion.betas[0]:.6f}, {diffusion.betas[-1]:.6f}]")
print("✅ U-Net + Diffusion ready!")

---

## 📊 Download Data

### Option 1: ModelNet40 (Quick - 500MB)

In [ ]:
# Download ModelNet40
!python scripts/download_data.py --dataset modelnet40

# Check downloaded data
!ls -lh data/ModelNet40/ 2>/dev/null || echo "Data downloading... Check scripts/download_data.py for manual instructions"

### Option 2: ShapeNet (Manual)

Để download ShapeNet:
1. Đăng ký tại https://shapenet.org/
2. Download ShapeNetCore.v2
3. Upload lên Google Drive
4. Mount Drive và copy data

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy ShapeNet data (nếu đã có trong Drive)
# !cp -r /content/drive/MyDrive/ShapeNetCore.v2 ./data/

---

## 🎨 Advanced Demo: Real ModelNet40 Mesh

In [ ]:
# Load a real mesh from ModelNet40
import trimesh
import glob
from pathlib import Path

# Find first available chair mesh
chair_meshes = glob.glob("data/ModelNet40/chair/train/*.off")
if chair_meshes:
    mesh_path = chair_meshes[0]
    print(f"📦 Loading: {Path(mesh_path).name}")
    
    # Load mesh
    mesh = trimesh.load(mesh_path, force='mesh')
    print(f"Mesh: {len(mesh.vertices)} vertices, {len(mesh.faces)} faces")
    
    # Convert to SDF
    sdf_real = mesh_to_sdf_simple(mesh, resolution=64)
    print(f"SDF shape: {sdf_real.shape}")
    
    # Wavelet transform
    sparse_real = sdf_to_sparse_wavelet(sdf_real, threshold=0.05)
    total = 64 ** 3
    non_zero = len(sparse_real['features'])
    sparsity_real = 100 * (1 - non_zero / total)
    
    print(f"Sparse indices: {sparse_real['indices'].shape}")
    print(f"Sparsity: {sparsity_real:.1f}%")
    print(f"Compression: {total / non_zero:.1f}x")
    
    # Reconstruct
    sdf_real_recon = sparse_wavelet_to_sdf(sparse_real)
    mse_real = np.mean((sdf_real - sdf_real_recon) ** 2)
    print(f"Reconstruction MSE: {mse_real:.6f}")
else:
    print("⚠️  No chair meshes found. Run download cell first!")

### Visualize Real Mesh Pipeline

In [ ]:
if chair_meshes:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(f'Real Mesh Pipeline: {Path(mesh_path).name}', fontsize=16, fontweight='bold')
    
    # Row 1: Different SDF slices
    for i, slice_idx in enumerate([16, 32, 48]):
        axes[0, i].imshow(sdf_real[slice_idx, :, :], cmap='RdBu', vmin=-1, vmax=1)
        axes[0, i].set_title(f'SDF Slice {slice_idx}/64')
        axes[0, i].axis('off')
    
    # Row 2: Reconstruction analysis
    axes[1, 0].imshow(sdf_real_recon[32, :, :], cmap='RdBu', vmin=-1, vmax=1)
    axes[1, 0].set_title('Reconstructed SDF')
    axes[1, 0].axis('off')
    
    # Error map
    error_real = np.abs(sdf_real - sdf_real_recon)
    axes[1, 1].imshow(error_real[32, :, :], cmap='hot')
    axes[1, 1].set_title(f'Error (MSE={mse_real:.6f})')
    axes[1, 1].axis('off')
    
    # Sparsity visualization
    sparse_viz_real = np.zeros((64, 64))
    for idx in sparse_real['indices']:
        if idx[2] == 32:
            sparse_viz_real[idx[0], idx[1]] += 1
    axes[1, 2].imshow(sparse_viz_real, cmap='hot')
    axes[1, 2].set_title(f'Sparse Coeffs ({sparsity_real:.1f}% sparse)')
    axes[1, 2].axis('off')
    
    plt.tight_layout()
    plt.show()
    print(f"✅ Real mesh pipeline complete! Compression: {total / non_zero:.1f}x")

### Multi-view Rendering (Optional)

In [ ]:
# Render multiple views of the mesh
# Note: Requires display, may not work in headless Colab
if chair_meshes:
    try:
        # Simple multi-view using matplotlib 3D
        from mpl_toolkits.mplot3d import Axes3D
        from mpl_toolkits.mplot3d.art3d import Poly3DCollection
        
        fig = plt.figure(figsize=(16, 4))
        
        # 4 different viewing angles
        angles = [
            (30, 45),   # Front-right
            (30, 135),  # Back-right
            (30, 225),  # Back-left
            (30, 315),  # Front-left
        ]
        
        for i, (elev, azim) in enumerate(angles):
            ax = fig.add_subplot(1, 4, i+1, projection='3d')
            
            # Create mesh collection
            mesh_collection = Poly3DCollection(
                mesh.vertices[mesh.faces], 
                alpha=0.7, 
                facecolor='cyan', 
                edgecolor='navy',
                linewidths=0.1
            )
            ax.add_collection3d(mesh_collection)
            
            # Set limits
            scale = mesh.vertices.max()
            ax.set_xlim([-scale, scale])
            ax.set_ylim([-scale, scale])
            ax.set_zlim([-scale, scale])
            
            # Set view angle
            ax.view_init(elev=elev, azim=azim)
            ax.set_title(f'View {i+1} ({azim}°)')
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_zlabel('Z')
        
        plt.tight_layout()
        plt.show()
        print("✅ Multi-view rendering complete!")
        
    except Exception as e:
        print(f"⚠️  Multi-view rendering failed: {e}")
        print("This is OK - rendering requires display capabilities.")

---

## 🎨 Visualize Pipeline

In [ ]:
# Visualize complete pipeline
# Note: visualize_results.py cần được tạo trước
# Hoặc dùng code đơn giản dưới đây:

import matplotlib.pyplot as plt
from data.wavelet_utils import WaveletTransform3D
import numpy as np

# Simple visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('WaveMesh-Diff Pipeline Overview', fontsize=16, fontweight='bold')

# 1. Input SDF
axes[0, 0].imshow(sdf[16, :, :], cmap='RdBu')
axes[0, 0].set_title('1. Input SDF (slice)')
axes[0, 0].axis('off')

# 2. Wavelet Coefficients (visualize sparsity)
sparse_indices = sparse_data['indices']
sparse_viz = np.zeros((32, 32))
for idx in sparse_indices:
    if idx[2] == 16:  # Same slice
        sparse_viz[idx[0], idx[1]] += 1
axes[0, 1].imshow(sparse_viz, cmap='hot')
axes[0, 1].set_title(f'2. Sparse Wavelet ({sparsity:.1f}% sparse)')
axes[0, 1].axis('off')

# 3. Reconstructed SDF
axes[1, 0].imshow(sdf_recon[16, :, :], cmap='RdBu')
axes[1, 0].set_title('3. Reconstructed SDF')
axes[1, 0].axis('off')

# 4. Reconstruction Error
error = np.abs(sdf - sdf_recon)
axes[1, 1].imshow(error[16, :, :], cmap='Reds')
axes[1, 1].set_title(f'4. Error (MSE={mse:.6f})')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()
print("✅ Pipeline visualization complete!")

---

## 🏋️ Training Example (Conceptual)

⚠️ **Lưu ý:** Training đầy đủ cần nhiều thời gian và GPU. Xem `ROADMAP.md` để có code đầy đủ.

---

## 🔄 End-to-End Pipeline Demo

In [ ]:
# Complete pipeline: Mesh → SDF → Wavelet → U-Net → Diffusion
print("🔄 End-to-End Pipeline Demo\n" + "="*60)

# Step 1: Input (use simple box for demo)
print("Step 1: Create input mesh")
demo_mesh = trimesh.creation.box(extents=[1, 1, 1])
print(f"  ✅ Mesh: {len(demo_mesh.vertices)} vertices")

# Step 2: Convert to SDF
print("\nStep 2: Convert to SDF")
demo_sdf = mesh_to_sdf_simple(demo_mesh, resolution=16)  # Small for speed
print(f"  ✅ SDF: {demo_sdf.shape}")

# Step 3: Wavelet transform
print("\nStep 3: Sparse wavelet representation")
demo_sparse = sdf_to_sparse_wavelet(demo_sdf, threshold=0.05)
print(f"  ✅ Sparse: {demo_sparse['indices'].shape[0]} coefficients")

# Step 4: Prepare for U-Net (convert to dense for demo)
print("\nStep 4: Prepare batch for U-Net")
# In real training, we'd use sparse tensor directly
# For demo, we'll use a small dense grid
demo_input = torch.randn(1, 1, 16, 16, 16)  # (B, C, D, H, W)
print(f"  ✅ Input: {demo_input.shape}")

# Step 5: U-Net denoising
print("\nStep 5: U-Net forward pass")
demo_unet = WaveMeshUNet(
    in_channels=1,
    encoder_channels=[8, 16],
    decoder_channels=[16, 8],
    time_emb_dim=64,
    use_attention=False
)
timesteps_demo = torch.tensor([500])  # Middle timestep
demo_output = demo_unet(demo_input, timesteps_demo, context=None)
print(f"  ✅ Output: {demo_output.shape}")

# Step 6: Diffusion denoising
print("\nStep 6: Diffusion sampling (conceptual)")
demo_diffusion = GaussianDiffusion(timesteps=100, beta_schedule='linear')
print(f"  ✅ Diffusion ready: {demo_diffusion.timesteps} steps")

print("\n" + "="*60)
print("✅ Complete pipeline working!")
print("\n📖 For full training loop, see ROADMAP.md")
print("   • Dataset loader for ModelNet40/ShapeNet")
print("   • Training with multi-view conditioning")
print("   • Evaluation metrics (Chamfer distance, F-score)")
print("   • Checkpoint saving/loading")

---

## ⚡ Performance Benchmarks

In [ ]:
import time

print("⚡ Performance Benchmarks")
print("="*60)

# Benchmark 1: Wavelet Transform
print("\n1. Wavelet Transform Speed")
resolutions = [16, 32, 64]
for res in resolutions:
    test_sdf = np.random.randn(res, res, res)
    
    start = time.time()
    test_sparse = sdf_to_sparse_wavelet(test_sdf, threshold=0.01)
    elapsed = time.time() - start
    
    sparsity = 100 * (1 - len(test_sparse['features']) / (res**3))
    print(f"  {res}³: {elapsed*1000:.1f}ms ({sparsity:.1f}% sparse)")

# Benchmark 2: U-Net Inference
print("\n2. U-Net Inference Speed")
test_unet = WaveMeshUNet(
    in_channels=1,
    encoder_channels=[16, 32],
    decoder_channels=[32, 16],
    time_emb_dim=64
)
test_unet.eval()

for res in [8, 16, 32]:
    test_input = torch.randn(1, 1, res, res, res)
    test_t = torch.tensor([100])
    
    # Warmup
    with torch.no_grad():
        _ = test_unet(test_input, test_t)
    
    # Benchmark
    start = time.time()
    with torch.no_grad():
        _ = test_unet(test_input, test_t)
    elapsed = time.time() - start
    
    params = sum(p.numel() for p in test_unet.parameters())
    print(f"  {res}³: {elapsed*1000:.1f}ms ({params:,} params)")

# Benchmark 3: Memory Usage
print("\n3. Memory Comparison")
for res in [32, 64]:
    dense_mb = (res**3 * 4) / (1024**2)  # float32
    
    test_sdf = np.random.randn(res, res, res)
    test_sparse = sdf_to_sparse_wavelet(test_sdf, threshold=0.01)
    sparse_mb = (len(test_sparse['features']) * 4) / (1024**2)
    
    compression = dense_mb / sparse_mb if sparse_mb > 0 else float('inf')
    print(f"  {res}³: Dense={dense_mb:.2f}MB, Sparse={sparse_mb:.2f}MB ({compression:.1f}x)")

print("\n" + "="*60)
print("✅ Benchmarks complete!")
print("\n💡 Tips for faster training:")
print("  • Use mixed precision (torch.cuda.amp)")
print("  • Install spconv for GPU sparse ops")
print("  • Reduce resolution during development (32³ → 16³)")
print("  • Enable gradient checkpointing for large models")

In [ ]:
# Training loop (conceptual)
# Xem ROADMAP.md để có implementation đầy đủ

from models import create_multiview_encoder, WaveMeshUNet, GaussianDiffusion
import torch

# 1. Prepare models
encoder = create_multiview_encoder(preset='small')
unet = WaveMeshUNet(
    in_channels=1,
    encoder_channels=[16, 32, 64],
    decoder_channels=[64, 32, 16],
    time_emb_dim=128,
    context_dim=384
)
diffusion = GaussianDiffusion(timesteps=1000)

# 2. Optimizer
optimizer = torch.optim.AdamW([
    {'params': encoder.parameters(), 'lr': 1e-5},
    {'params': unet.parameters(), 'lr': 1e-4}
])

print("✅ Training setup ready!")
print(f"Encoder params: {sum(p.numel() for p in encoder.parameters()):,}")
print(f"U-Net params: {sum(p.numel() for p in unet.parameters()):,}")
print("\n📖 See ROADMAP.md for full training code with:")
print("  • Dataset implementation")
print("  • Training loop")
print("  • Evaluation metrics")
print("  • Checkpointing")

---

## 📚 Next Steps

1. **Đọc Documentation:**
   - [README.md](https://github.com/HoangNguyennnnnnn/WaveMeshDf/blob/main/README.md) - Project overview
   - [QUICKSTART.md](https://github.com/HoangNguyennnnnnn/WaveMeshDf/blob/main/QUICKSTART.md) - Quick start guide
   - [ROADMAP.md](https://github.com/HoangNguyennnnnnn/WaveMeshDf/blob/main/ROADMAP.md) - Training roadmap
   - [ARCHITECTURE.md](https://github.com/HoangNguyennnnnnn/WaveMeshDf/blob/main/ARCHITECTURE.md) - Technical details

2. **Train Your Own Model:**
   - Implement dataset loader (see ROADMAP.md)
   - Set up training loop with multi-view conditioning
   - Add evaluation metrics (Chamfer distance, F-score)
   - Save/load checkpoints

3. **Improve Quality:**
   - Use DINOv2 pretrained encoder (`pip install transformers`)
   - Implement classifier-free guidance (CFG)
   - Add EMA for stable training
   - Use AdaLN for better conditioning

4. **Scale Up:**
   - Install spconv for GPU sparse operations
   - Use mixed precision training (`torch.cuda.amp`)
   - Increase resolution (64³ or 128³)
   - Train on ShapeNet (50GB, 55 categories)

---

## 🐛 Troubleshooting

### "ModuleNotFoundError: No module named 'pywt'"
```python
!pip install PyWavelets
```

### "ModuleNotFoundError: No module named 'rtree'"
```python
!pip install rtree
```

### "ValueError: too many values to unpack (expected 2)"
Lỗi này xảy ra khi dùng API cũ. `sdf_to_sparse_wavelet()` trả về **dictionary**, không phải tuple:
```python
# ❌ Sai:
coeffs, coords = sdf_to_sparse_wavelet(sdf)

# ✅ Đúng:
sparse_data = sdf_to_sparse_wavelet(sdf, threshold=0.01)
print(sparse_data['indices'].shape)
print(sparse_data['features'].shape)
```

### "FileNotFoundError: No such file or directory: 'data/ModelNet40/train'"
ModelNet40 structure đã được sửa. Mỗi category có train/test subfolders riêng. Script đã được update để xử lý đúng structure này.

### "CUDA out of memory"
```python
# Giảm batch size hoặc resolution
batch_size = 2
resolution = 16
```

### "transformers not available"
```python
!pip install transformers huggingface_hub
# Code sẽ tự động fallback sang CNN nếu không có
```

### "No module named 'skimage'"
```python
!pip install scikit-image
```

### Rendering fails in Colab
Multi-view rendering có thể fail do headless environment. Đây là bình thường - code vẫn hoạt động mà không cần visualization.

Xem đầy đủ tại [TROUBLESHOOTING.md](https://github.com/HoangNguyennnnnnn/WaveMeshDf/blob/main/TROUBLESHOOTING.md).

---

## 🎯 Summary

**What you learned:**
- ✅ Setup WaveMesh-Diff in Google Colab
- ✅ Test all 4 modules (Wavelet, U-Net, Diffusion, Multi-view)
- ✅ Convert mesh → SDF → sparse wavelet
- ✅ Visualize pipeline and reconstruction quality
- ✅ Understand memory savings from sparsity (60-90% compression)
- ✅ Run complete end-to-end pipeline
- ✅ Benchmark performance

**Next steps:**
- 📖 Read ROADMAP.md for training implementation
- 🏋️ Implement dataset loader for ModelNet40
- 🚀 Train your first diffusion model
- 🎨 Generate 3D meshes from multi-view images!

---

**Happy 3D Generation! 🎨**

Questions? Open an issue: https://github.com/HoangNguyennnnnnn/WaveMeshDf/issues